# Preprocessing

### Loading

In [2]:
import os
import numpy as np
import pandas as pd
from skimage import filters, color
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import rsatoolbox
from rsatoolbox.data import Dataset
from rsatoolbox.rdm import calc_rdm
from scipy.spatial.distance import cosine
from scipy.stats import spearmanr, t
from scipy.cluster.hierarchy import linkage, dendrogram
from scipy.spatial.distance import pdist, squareform
from cliffs_delta import cliffs_delta
from sklearn.preprocessing import normalize
from bisect import bisect_right, bisect_left
from scipy.spatial.distance import pdist

### Constants

In [ ]:
models = ["conch", "plip", "prov", "quiltnet", "resnet", "uni", "virchow", "keep", "musk"]
model_tags = ['CONCH', 'PLIP', 'Prov-Gigapath', 'QuiltNet', 'ResNet50', 'UNI', 'Virchow', 'KEEP','MUSK']
display_names = model_tags
subtypes = ['brca', 'coad', 'luad', 'lusc']
num_slides = 50
num_patches = 50
n_batches = 5

### Splitting Into 5 Batches of 50 slides/50 patches each for validation

In [ ]:
base_path = "/lotterlab/users/vmishra/"
output_dir = "/lotterlab/users/vmishra/batched_embeddings_normalized"
os.makedirs(output_dir, exist_ok=True)
slides_per_batch = num_slides
patches_per_slide = num_patches

for subtype in subtypes:
    for model in models:
        file_path = os.path.join(base_path, f"{subtype}_embeddings_{model}_normalized.npy")
        print(f"Processing {file_path}...")

        embeddings = np.load(file_path)
        num_total = 250 * 250
        embedding_dim = embeddings.shape[1]
        assert embeddings.shape[0] == num_total, f"Unexpected shape for {file_path}"

        embeddings = embeddings.reshape(250, 250, embedding_dim)

        for batch_idx in range(5):
            start_slide = batch_idx * slides_per_batch
            end_slide = (batch_idx + 1) * slides_per_batch

            batch = embeddings[start_slide:end_slide, :patches_per_slide, :]
            batch = batch.reshape(-1, embedding_dim)

            batch_filename = f"{subtype}_{model}_batch{batch_idx+1}_normalized.npy"
            batch_path = os.path.join(output_dir, batch_filename)
            np.save(batch_path, batch)
            print(f"Saved batch {batch_idx+1} to {batch_path}")

In [ ]:
input_dir = "/lotterlab/users/vmishra/batched_embeddings_normalized"
rdm_output_dir = "/lotterlab/users/vmishra/rdms_normalized"
os.makedirs(rdm_output_dir, exist_ok=True)

for model in models:
    for batch_idx in range(1, n_batches + 1):
        print(f"Processing RDM for model: {model}, batch: {batch_idx}")

        embeddings_list = []
        labels = []

        for subtype in subtypes:
            file_path = os.path.join(input_dir, f"{subtype}_{model}_batch{batch_idx}_normalized.npy")
            if not os.path.exists(file_path):
                print(f"File not found: {file_path}, skipping.")
                continue

            emb = np.load(file_path)
            embeddings_list.append(emb)
            labels.extend([subtype.upper()] * len(emb))

        if len(embeddings_list) < 4:
            print(f"Missing data for model {model}, batch {batch_idx}. Skipping.")
            continue

        embeddings = np.concatenate(embeddings_list, axis=0)

        dataset = Dataset(measurements=embeddings, obs_descriptors={'disease': labels})
        rdm = calc_rdm(dataset, method='euclidean')
        rdm_matrix = rdm.get_matrices()[0]

        save_path = os.path.join(rdm_output_dir, f"rdm_matrix_{model}_batch{batch_idx}_normalized.npy")
        np.save(save_path, rdm_matrix)
        print(f"Saved RDM to {save_path}")

# RDMs

In [ ]:
def make_plot(rdm, model_name):
    n = rdm.shape[0] / 4
    divider_positions = [n, 2*n, 3*n]
    diseases = ["BRCA", "COAD", "LUAD", "LUSC"]

    plt.figure(figsize=(10, 8))
    ax = sns.heatmap(rdm, cmap='Blues', annot=False, cbar=True)
    ax.collections[0].colorbar.set_label("Normalized Distance", fontsize=12)
    plt.title(model_name, fontweight='bold', fontsize=20)
    plt.xticks([n // 2, n + n // 2, n + n + n // 2, n + n + n + n // 2],
               diseases, rotation=0, ha='center', fontsize=12, fontweight='bold')
    plt.yticks([n // 2, n + n // 2, n + n + n // 2, n + n + n + n // 2],
               diseases, rotation=0, fontsize=12, fontweight='bold')

    # Add divider lines between disease types
    for pos in divider_positions:
        ax.axhline(pos, color='black', linewidth=1.5)
        ax.axvline(pos, color='black', linewidth=1.5)

    plt.savefig("rdm_matrixv2_normalized-" + model_name + ".png", dpi=300, bbox_inches='tight')

for m, t in zip(models, model_tags):
    print(m)
    rdm_mat = np.load("/lotterlab/users/vmishra/rdms_normalized/rdm_matrix_" + m + "_batch1_normalized.npy")
    rdm_mat = rdm_mat / rdm_mat.max()
    make_plot(rdm_mat, t)

# Spearman and Cosine Similarity Heatmap

In [ ]:
n_models = len(models)
rdm_dir = "/lotterlab/users/vmishra/rdms_normalized"

cosine_mean = np.zeros((n_models, n_models))
cosine_ci_lower = np.zeros((n_models, n_models))
cosine_ci_upper = np.zeros((n_models, n_models))

spearman_mean = np.zeros((n_models, n_models))
spearman_ci_lower = np.zeros((n_models, n_models))
spearman_ci_upper = np.zeros((n_models, n_models))

def get_range(values):
    return np.mean(values), np.min(values), np.max(values)

for i, model_i in enumerate(models):
    for j, model_j in enumerate(models):
        cosine_vals = []
        spearman_vals = []

        for batch in range(1, n_batches + 1):
            path_i = os.path.join(rdm_dir, f"rdm_matrix_{model_i}_batch{batch}_normalized.npy")
            path_j = os.path.join(rdm_dir, f"rdm_matrix_{model_j}_batch{batch}_normalized.npy")
            rdm_i = np.load(path_i)
            rdm_j = np.load(path_j)

            tri_i = rdm_i[np.triu_indices_from(rdm_i, k=1)]
            tri_j = rdm_j[np.triu_indices_from(rdm_j, k=1)]

            cos_sim = 1 - cosine(tri_i, tri_j)
            spearman_corr, _ = spearmanr(tri_i, tri_j)

            cosine_vals.append(cos_sim)
            spearman_vals.append(spearman_corr)

        # Cosine
        mean_cos, ci_cos_low, ci_cos_high = get_range(cosine_vals)
        cosine_mean[i, j] = mean_cos
        cosine_ci_lower[i, j] = ci_cos_low
        cosine_ci_upper[i, j] = ci_cos_high

        # Spearman
        mean_spear, ci_spear_low, ci_spear_high = get_range(spearman_vals)
        spearman_mean[i, j] = mean_spear
        spearman_ci_lower[i, j] = ci_spear_low
        spearman_ci_upper[i, j] = ci_spear_high

# Desired model order and display names
desired_order = ['uni', 'virchow', 'prov', 'conch', 'plip', 'quiltnet', 'keep', 'musk', 'resnet']
display_names = ['UNI', 'Virchow', 'Prov-GigaPath', 'CONCH', 'PLIP', 'QuiltNet', 'KEEP', 'MUSK', 'ResNet-50']
reorder_idx = [models.index(m) for m in desired_order]

# Reorder all matrices
cosine_mean_r = cosine_mean[reorder_idx, :][:, reorder_idx]
cosine_ci_lower_r = cosine_ci_lower[reorder_idx, :][:, reorder_idx]
cosine_ci_upper_r = cosine_ci_upper[reorder_idx, :][:, reorder_idx]

spearman_mean_r = spearman_mean[reorder_idx, :][:, reorder_idx]
spearman_ci_lower_r = spearman_ci_lower[reorder_idx, :][:, reorder_idx]
spearman_ci_upper_r = spearman_ci_upper[reorder_idx, :][:, reorder_idx]

def format_annot(mean_mat, lower_mat, upper_mat):
    n = mean_mat.shape[0]
    annot = np.empty((n, n), dtype=object)
    for i in range(n):
        for j in range(n):
            m = mean_mat[i, j]
            l = lower_mat[i, j]
            u = upper_mat[i, j]
            annot[i, j] = f"{m:.3f}\n[{l:.3f}–{u:.3f}]"
    return annot

cosine_annot = format_annot(cosine_mean_r, cosine_ci_lower_r, cosine_ci_upper_r)
spearman_annot = format_annot(spearman_mean_r, spearman_ci_lower_r, spearman_ci_upper_r)

# Plot cosine similarity
plt.figure(figsize=(10, 8))
sns.heatmap(cosine_mean_r, annot=cosine_annot, fmt="", xticklabels=display_names, yticklabels=display_names,
            cmap="Reds", vmin=0.85, vmax=1, cbar_kws={"label": "Cosine Similarity"})
plt.title("Cosine Similarity Between Model RDMs (Normalized)\nMean [Range Across 5 Batches]")
plt.tight_layout()
plt.savefig("cosineHeatmap_normalized.pdf", format="pdf", bbox_inches="tight")
plt.show()

# Plot Spearman correlation
plt.figure(figsize=(10, 8))
sns.heatmap(spearman_mean_r, annot=spearman_annot, fmt="", xticklabels=display_names, yticklabels=display_names,
            cmap="Reds", vmin=0, vmax=1, cbar_kws={"label": "Spearman Correlation"})
plt.title("Spearman Correlation Between Model RDMs (Normalized)\nMean [Range Across 5 Batches]")
plt.tight_layout()
plt.savefig("spearmanHeatmap_normalized.pdf", format="pdf", bbox_inches="tight")
plt.show()

# Mean Spearman correlation

In [ ]:
n_models = len(display_names)

average_spearman_corr = (np.sum(spearman_mean_r, axis=1) - np.diag(spearman_mean_r)) / (n_models - 1)

df_similarity = pd.DataFrame({
    "Model": display_names,
    "Average Spearman Correlation": average_spearman_corr
})

df_similarity

# Dendrograms (Change when above values are run)

In [ ]:
n_models = len(models)
spearman_mean = np.ones((n_models, n_models))

spearman_mean[0, 1:] = [0.279, 0.41, 0.4, 0.223, 0.195, 0.122]
spearman_mean[1, 2:] = [0.466, 0.391, 0.43, 0.371, 0.136]
spearman_mean[2, 3:] = [0.505, 0.54, 0.515, 0.181]
spearman_mean[3, 4:] = [0.485, 0.734, 0.222]
spearman_mean[4, 5:] = [0.384, 0.151]
spearman_mean[5, 6] = 0.192

for i in range(1, n_models):
    for j in range(i):
        spearman_mean[i, j] = spearman_mean[j, i]

In [ ]:
spearman_mean

In [ ]:
spearman_distances = 1 - spearman_mean

# Convert to condensed form
spearman_condensed = squareform(spearman_distances, checks=False)

# Hierarchical Clustering (Ward's method)
linkage_spearman = linkage(spearman_condensed, method="ward")

In [ ]:
thresholds = [0.1, 0.33, 0.55, 0.62, 0.70, 0.81, 1.00]

# Get Seaborn color palette with one more color than thresholds
colors = sns.color_palette("deep", len(thresholds) + 1)
colors = ['blue', 'orange', 'green', 'red', 'purple', 'brown', 'gray']

# Custom link color function based on thresholds
def link_color_func(link_id):
    print(link_id)
    #print(link_id)
#     if link_id < linkage_spearman.shape[0]:
#         print(link_id)
    row = link_id - linkage_spearman.shape[0] - 1
    dist = linkage_spearman[row, 2]  # height of the node
    for i, t in enumerate(thresholds):
        if dist < t:
            return colors[i]
    return colors[-1]
#     else:
#         return 'black'

In [ ]:
fig, ax = plt.subplots()

#plt.figure(figsize=(10, 5))
ct = [1, ]
dendrogram(linkage_spearman, labels=models, leaf_rotation=90, leaf_font_size=12, color_threshold=0.82) #, link_color_func=link_color_func)
plt.title("Hierarchical Clustering of Model RDM Similarity (Normalized)", fontweight='bold')
#plt.xlabel("Models")
plt.ylabel("Distance")
plt.tight_layout()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.ylabel('Ward Distance', fontsize=12) #, fontweight='bold')

plt.savefig('ClusteringSpearman_normalized_v2.pdf')

In [ ]:
linkage_spearman.shape

In [ ]:
linkage_spearman

In [ ]:
colors[0]

In [ ]:
colors

In [ ]:
models = ["UNI", "Virchow", "Prov-GigaPath", "CONCH", "PLIP", "QuiltNet", "ResNet50"]
# just copied from plot since taking so long
n_models = len(models)
cosine_mean = np.ones((n_models, n_models))

cosine_mean[0, 1:] = [0.939, 0.966, 0.946, 0.924, 0.922, 0.883]
cosine_mean[1, 2:] = [0.964, 0.942, 0.936, 0.933, 0.879]
cosine_mean[2, 3:] = [0.963, 0.957, 0.958, 0.905]
cosine_mean[3, 4:] = [0.947, 0.937, 0.88]
cosine_mean[4, 5:] = [0.972, 0.882]
cosine_mean[5, 6] = 0.878

for i in range(1, n_models):
    for j in range(i):
        cosine_mean[i, j] = cosine_mean[j, i]

In [ ]:
cosine_mean

In [ ]:
cosine_distances = 1 - cosine_mean

# Convert to condensed form
cosine_condensed = squareform(cosine_distances, checks=False)

# Hierarchical Clustering (Ward's method)
linkage_cosine = linkage(cosine_condensed, method="ward")

In [ ]:
fig, ax = plt.subplots()

#plt.figure(figsize=(10, 5))
ct = [1, ]
dendrogram(linkage_cosine, labels=models, leaf_rotation=90, leaf_font_size=12, color_threshold=0.14) #, link_color_func=link_color_func)
plt.title("Hierarchical Clustering of Model RDM Similarity (Normalized)", fontweight='bold')
#plt.xlabel("Models")
plt.ylabel("Distance")
plt.tight_layout()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.ylabel('Ward Distance', fontsize=12) #, fontweight='bold')

plt.savefig('ClusteringCosine_normalized_v2.pdf')

# Slide Specificity

In [ ]:
embedding_dir = "/lotterlab/users/vmishra/batched_embeddings_normalized"
def calculate_distances(embeddings, num_slides, patches_per_slide):
    intra_distances = []
    inter_distances = []

    for slide_idx in range(num_slides):
        slide_indices = np.arange(slide_idx * patches_per_slide, (slide_idx + 1) * patches_per_slide)
        other_indices = np.setdiff1d(np.arange(embeddings.shape[0]), slide_indices)

        slide_distances = pdist(embeddings[slide_indices], metric='euclidean')
        intra_distances.extend(slide_distances)

        for other_idx in other_indices:
            inter_distances.extend(np.linalg.norm(embeddings[slide_indices] - embeddings[other_idx], axis=1))

    return np.array(intra_distances), np.array(inter_distances)

def efficient_cliffs_delta(a, b):
    a = np.sort(a)
    b = np.sort(b)
    m, n = len(a), len(b)
    more = sum(bisect_right(b, x) for x in a)
    less = sum(n - bisect_left(b, x) for x in a)
    delta = (more - less) / (m * n)
    return delta

results = []

for model in tqdm(models, desc="Processing models"):
    delta_vals = []

    for batch in range(1, n_batches + 1):
        embeddings_list = []
        for subtype in subtypes:
            file_path = os.path.join(embedding_dir, f"{subtype}_{model}_batch{batch}_normalized.npy")
            emb = np.load(file_path)
            embeddings_list.append(emb)

        all_embeddings = np.vstack(embeddings_list)
        intra, inter = calculate_distances(all_embeddings, num_slides=num_slides * 4, patches_per_slide=patches_per_slide)

        delta_vals.append(efficient_cliffs_delta(intra, inter))

    delta_vals = np.array(delta_vals)

    results.append({
        "model": model,
        "display_name": display_names[models.index(model)],
        "cliffs_delta_mean": delta_vals.mean(),
        "cliffs_delta_min": delta_vals.min(),
        "cliffs_delta_max": delta_vals.max()
    })

results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by="cliffs_delta_mean", ascending=False).reset_index(drop=True)
results_df.to_csv("slide_specificity_normalized.csv", index=False)
print(results_df)

# Disease Specificity

In [ ]:
embedding_dir = "/lotterlab/users/vmishra/batched_embeddings_normalized"
def efficient_cliffs_delta(a, b):
    a = np.sort(a)
    b = np.sort(b)
    m, n = len(a), len(b)
    more = sum(bisect_right(b, x) for x in a)
    less = sum(n - bisect_left(b, x) for x in a)
    delta = (more - less) / (m * n)
    return delta

results = []

for model in tqdm(models, desc="Processing models"):
    delta_vals = []

    for batch in range(1, n_batches + 1):
        disease_embeddings = {}
        for subtype in subtypes:
            file_path = os.path.join(embedding_dir, f"{subtype}_{model}_batch{batch}_normalized.npy")
            disease_embeddings[subtype] = np.load(file_path)

        intra_distances = []
        for subtype in subtypes:
            emb = disease_embeddings[subtype]
            dists = pdist(emb, metric='euclidean')
            intra_distances.extend(dists)

        inter_distances = []
        for i in range(len(subtypes)):
            for j in range(i + 1, len(subtypes)):
                emb1 = disease_embeddings[subtypes[i]]
                emb2 = disease_embeddings[subtypes[j]]
                diffs = np.linalg.norm(emb1[:, None, :] - emb2[None, :, :], axis=2).flatten()
                inter_distances.extend(diffs)

        intra = np.array(intra_distances)
        inter = np.array(inter_distances)

        delta_vals.append(efficient_cliffs_delta(intra, inter))

    delta_vals = np.array(delta_vals)

    results.append({
        "model": model,
        "display_name": display_names[models.index(model)],
        "cliffs_delta_mean": delta_vals.mean(),
        "cliffs_delta_min": delta_vals.min(),
        "cliffs_delta_max": delta_vals.max()
    })

results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by="cliffs_delta_mean", ascending=False).reset_index(drop=True)
results_df.to_csv("disease_specificity_normalized.csv", index=False)
print(results_df)

# Spectral Analysis

In [ ]:
models = ["UNI", "Virchow", "Prov-GigaPath", "CONCH", "PLIP", "QuiltNet", "KEEP", "MUSK", "ResNet-50"]
model_file_keys = {
    "UNI": "uni",
    "Virchow": "virchow",
    "Prov-GigaPath": "prov",
    "CONCH": "conch",
    "PLIP": "plip",
    "QuiltNet": "quiltnet",
    "KEEP": "keep",
    "MUSK": "musk",
    "ResNet-50": "resnet"
}

vl_models = ["PLIP", "QuiltNet", "CONCH", "KEEP", "MUSK"]

embedding_dir = "/lotterlab/users/vmishra"

embeddings = {}

for model_name in models:
    model_key = model_file_keys[model_name]
    embeddings_list = []

    print(f"\nProcessing {model_name}...")
    for subtype in subtypes:
        file_path = os.path.join(embedding_dir, f"{subtype}_embeddings_{model_key}_normalized.npy")
        if not os.path.exists(file_path):
            print(f"Missing: {file_path}")
            continue

        emb = np.load(file_path)
        embeddings_list.append(emb)

    if len(embeddings_list) < 4:
        print(f"Skipping {model_name} due to missing data.")
        continue

    embeddings[model_name] = np.concatenate(embeddings_list, axis=0)
    embeddings[model_name] -= embeddings[model_name].mean(axis=0)

spectra = {}
for model_name in models:
    print(model_name)
    U, S, Vt = np.linalg.svd(embeddings[model_name], full_matrices=False)
    normalized_spectrum = S / S.sum()
    spectra[model_name] = normalized_spectrum

for model_name in models:
    print(model_name, embeddings[model_name].shape)

normalized_spectrum.sum(axis=-1)

p_test = [0.25] + list(range(1, 101))
s_sums = np.zeros((len(models), len(p_test)))
for i, m in enumerate(models):
    n_feat = embeddings[m].shape[-1]
    for j, p in enumerate(p_test):
        cut_off = int(np.round(n_feat * p/100))
        s_sums[i, j] = spectra[m][:(cut_off+1)].sum()

fig, ax = plt.subplots()

for i, m in enumerate(models):
    line_style = '--' if m in vl_models else '-'
    plt.plot(p_test, s_sums[i], label=m, linewidth=2, linestyle=line_style)

plt.legend(prop={'weight':'bold', 'size': 11})
ax.set_ylim([0, 1.02])
ax.set_aspect(100, adjustable='box')
plt.xlabel('Percentage of Features', fontsize=12, fontweight='bold')
plt.ylabel('Singular Value Cumulative Sum', fontsize=12, fontweight='bold')
plt.title('SVD Spectral Analysis (Normalized Embeddings)', fontsize=14, fontweight='bold')

# Remove top and right spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.savefig('spectral_analysis_normalized.pdf', dpi=300, bbox_inches='tight', pad_inches=0.05)